In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime, timezone, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import re
file_path = './data/TAF/train/2022-09-04.csv'
fuser_df = pd.read_csv(file_path)
nan_s = fuser_df.isnull().sum()
print(nan_s[nan_s> 0])


Series([], dtype: int64)


In [2]:
taf_pattern = re.compile(
    r'^(?:PART\s+\d+(?:\s+OF(?:\s+\d+)?)?\s+)?'                           # Optional PART X OF Y
    r'(?:TAF\s+)*'                                               # Allow multiple TAF prefixes
    r'(?P<amended>(AMD)\s+)?'                                 # Optional AMD flag for amended reports
    r'(?P<corection>(COR)\s+)?'                                 # Optional COR flag for corrected reports
    r'(?P<station>\w{4})(?:\s+\w{4})?\s+'                                      # ICAO station code
    r'(?P<issue_datetime>\d{6}Z\s+)?'                             # Optional issue date and time
    r'(?P<validity>(?P<valid_start_day>\d{2})(?P<valid_start_hour>\d{2})/'
    r'(?P<valid_end_day>\d{2})(?P<valid_end_hour>\d{2}))\s+'      # Validity period
    r'(?P<wind>(VRB|\d{3})\d{2,3}(G\d{2,3})?[/|?]?(KT|MPS|KMH)?)?\s*' # Wind information
    r'(?P<visibility>\d{4}|CAVOK|9999|////|[0-9]+SM)?\s*'         # Visibility

    # Weather with optional intensity prefix (+ or -) and non-greedy match
    r'(?P<weather>(\+|-)?[A-Z]{2,6})?\s+'                         # Weather phenomena with optional intensity, followed by whitespace
    r'(?P<clouds>((FEW|SCT|BKN|OVC|NSC|VV|NCD|CLR|SKC|///)\d{3}(CB)?\s*)*)' # Cloud layers
    r'(?P<qnh>QNH\d{4}(INS|HPA)?)?\s*'                            # QNH (altimeter setting) in inches or hPa
    r'(?P<variable_wind>WND\s+\d{3}V\d{3})?\s*'                   # Variable wind direction

    # Max/Min Temperature capture with optional minus sign for temperatures
    r'(?P<max_temp>TX(?P<max_temp_value>M?\d{2})/'                # Max temperature with optional minus sign
    r'(?P<max_temp_day>\d{2})(?P<max_temp_hour>\d{2})Z\s*)?'      
    r'(?P<min_temp>TN(?P<min_temp_value>M?\d{2})/'                # Min temperature with optional minus sign
    r'(?P<min_temp_day>\d{2})(?P<min_temp_hour>\d{2})Z\s*)?'      
    r'(?P<probability>PROB(?P<prob_value>\d{2}))?\s*'             # Probability

    # Capture everything else in additional_sections
    r'(?P<additional_sections>.*?)$'                               # Capture all remaining parts as additional sections
)


In [3]:
def remove_duplicates_in_report(report):
    words = report.split()
    seen = set()
    result = []
    for word in words:
        if word not in seen:
            seen.add(word)
            result.append(word)
    return ' '.join(result)


In [4]:
def process_validity(data):
    # Ensure date_time is a datetime object in UTC for calculations
    base_date = pd.to_datetime(data['date_time'], utc=True)

    # Construct valid_start_date
    start_day = int(data['valid_start_day'])
    start_hour = int(data['valid_start_hour'])
    valid_start_date = base_date.replace(day=start_day, hour=start_hour)

    # If the start day is before the base date's day (new month rollover), adjust the month and year
    if valid_start_date < base_date:
        valid_start_date += timedelta(days=30)  # Adjust depending on month rollover logic
    if valid_start_date < base_date:
        valid_start_date += relativedelta(months=1)
        
    # Construct valid_end_date
    end_day = int(data['valid_end_day'])
    end_hour = int(data['valid_end_hour'])
    
    if end_hour == 24:
        end_hour = 0
        end_day += 1
        
    valid_end_date = base_date.replace(day=end_day, hour=end_hour)

    # Handle cases where end day may be in the following month
    if valid_end_date < valid_start_date:
        valid_end_date += timedelta(days=30)  # Adjust based on month rollover logic

    # Add results back to the data dictionary
    data['valid_start_date'] = valid_start_date
    data['valid_end_date'] = valid_end_date
    return data

In [5]:
def process_wind(data):
    """Processes the wind field, extracts direction and speed, and converts units to knots if necessary."""
    # Extract the wind field
    wind_str = data.get('wind')
    if wind_str is None:
        # Set default values for missing wind data
        data['wind_direction'] = 999  # 999 indicates variable or missing direction
        data['wind_speed_kt'] = None
        data['wind_gust_kt'] = None
        return data

    # Use regex to match and extract components of the wind string
    match = re.match(r'^(VRB|\d{3})?([/?]?\d{2,3})(G[/?]?\d{2,3})?(KT|MPS|KMH)?$', wind_str)
    if not match:
        # Set defaults for unrecognized wind patterns
        data['wind_direction'] = 999
        data['wind_speed_kt'] = None
        data['wind_gust_kt'] = None
        return data

    # Extract matched components
    direction, speed, gust, unit = match.groups()

    # Set direction to 999 if variable (VRB) or missing
    direction = 999 if direction == 'VRB' or direction is None else int(direction)
    speed = int(speed) if speed and speed.isdigit() else None
    gust = int(gust[1:]) if gust and gust[1:].isdigit() else None

    # Convert speeds to knots (KT) if necessary
    if unit == 'MPS':
        speed = round(speed * 1.94384) if speed else None  # MPS to KT
        gust = round(gust * 1.94384) if gust else None
    elif unit == 'KMH':
        speed = round(speed * 0.539957) if speed else None  # KMH to KT
        gust = round(gust * 0.539957) if gust else None

    # Update data dictionary with processed values
    data['wind_direction'] = direction
    data['wind_speed_kt'] = speed
    data['wind_gust_kt'] = gust
    return data


In [6]:
weather_mapping = {
    "DZ": 1,  # Drizzle
    "RA": 3,  # Rain
    "SN": 5,  # Snow
    "SG": 2,  # Snow Grains
    "IC": 2,  # Ice Crystals
    "PL": 3,  # Ice Pellets
    "GR": 6,  # Hail
    "GS": 4,  # Small Hail or Snow Pellets
    "UP": 2,  # Unknown Precipitation
    "BR": 1,  # Mist
    "FG": 3,  # Fog
    "FU": 4,  # Smoke
    "VA": 5,  # Volcanic Ash
    "DU": 3,  # Dust
    "SA": 4,  # Sand
    "HZ": 2,  # Haze
    "PY": 3,  # Spray
    "PO": 4,  # Dust/Sand Whirls
    "SQ": 5,  # Squall
    "FC": 6,  # Funnel Cloud (Tornado/Waterspout)
    "SS": 6,  # Sandstorm
    "DS": 6,  # Duststorm
    "SH": 4,  # Showers
    "TS": 5,  # Thunderstorm
    "FZ": 5   # Freezing
}

# Intensity multipliers
intensity_mapping = {
    "-": 0.5,  # Light intensity
    "+": 1.5,  # Heavy intensity
    None: 1.0  # Standard intensity
}

def process_weather(data):
    """Processes the 'weather' field in the data dictionary to assign a score based on intensity and type."""
    weather_str = data.get('weather', None)
    if not weather_str:
        data['weather_score'] = 0  # Default score if no weather data is present
        return data

    # Extract intensity and weather type
    match = re.match(r'^(\+|-)?([A-Z]{2,6})$', weather_str)
    if not match:
        data['weather_score'] = 0  # Default score for unrecognized patterns
        return data

    # Get the intensity and weather code
    intensity, weather_code = match.groups()

    # Calculate score based on weather code and intensity
    base_score = weather_mapping.get(weather_code, 0)  # Get base score for weather type
    multiplier = intensity_mapping.get(intensity, 1.0)  # Apply multiplier based on intensity
    score = base_score * multiplier

    # Update data dictionary with calculated weather score
    data['weather_score'] = score
    return data


In [7]:
cloud_cover_dict = {
    "SKC": 0,  # Sky Clear
    "CLR": 0,  # Clear
    "NSC": 0,  # No Significant Clouds
    "NCD": 0,  # No Cloud Detected
    "FEW": 1,  # Few (1/8 to 2/8 sky cover)
    "SCT": 3,  # Scattered (3/8 to 4/8 sky cover)
    "BKN": 5,  # Broken (5/8 to 7/8 sky cover)
    "OVC": 8,  # Overcast (8/8 sky cover)
    "VV": 9    # Vertical Visibility (obscured sky, treated as full overcast)
}

def parse_cloud_layers(cloud_layers):
    """Convert cloud layer codes to structured data for modeling."""
    parsed_layers = []
    for layer in cloud_layers:
        # Match cloud layer code, altitude, and CB flag if present
        match = re.match(r"([A-Z]{3})(\d{3})?(CB)?", layer)
        if match:
            cloud_code, altitude, cumulonimbus = match.groups()
            sky_cover = cloud_cover_dict.get(cloud_code, None)  # Get sky cover value
            altitude_ft = int(altitude) * 100 if altitude else None  # Convert altitude to feet if present
            cb_flag = 1 if cumulonimbus else 0  # Cumulonimbus flag (1 for CB, 0 otherwise)

            # Append the structured cloud data
            parsed_layers.append({
                "sky_cover": sky_cover,           # Numerical sky cover level
                "altitude_ft": altitude_ft,       # Altitude in feet
                "cumulonimbus": cb_flag           # Cumulonimbus presence (1 or 0)
            })
    return parsed_layers

In [ ]:
def process_qnh(data):
    """
    Process the 'qnh' field to a unified unit in HPA.
    If the unit is in inches (INS), convert it to HPA.
    If the unit is missing, set it as None.
    """
    qnh_str = data.get('qnh', None)
    
    if qnh_str is None:
        data['qnh_hpa'] = None
        return data

    # Match QNH format
    match = re.match(r'QNH(\d{4})(INS|HPA)?', qnh_str)
    if match:
        qnh_value, unit = match.groups()
        qnh_value = int(qnh_value)

        # Convert INS to HPA if necessary
        if unit == "INS":
            qnh_value = round(qnh_value * 33.8639)  # 1 inHg = 33.8639 hPa
        data['qnh_hpa'] = qnh_value
    else:
        data['qnh_hpa'] = None  # Default for unrecognized patterns

    return data

In [9]:
def process_variable_wind(data):
    """
    Process the 'variable_wind' field to extract the variable wind directions.
    If the wind is variable, extract starting and ending directions in degrees.
    If the format is unrecognized or missing, set values to None.
    """
    variable_wind_str = data.get('variable_wind', None)
    
    if variable_wind_str is None:
        data['variable_wind_from'] = None
        data['variable_wind_to'] = None
        return data

    # Match variable wind format WND XXXVYYY
    match = re.match(r'WND\s+(\d{3})V(\d{3})', variable_wind_str)
    if match:
        from_dir, to_dir = match.groups()
        data['variable_wind_from'] = int(from_dir)
        data['variable_wind_to'] = int(to_dir)
    else:
        # If the format is not recognized, set to None
        data['variable_wind_from'] = None
        data['variable_wind_to'] = None

    return data

In [ ]:
def convert_temp_to_datetime(data):
    """
    Converts max_temp_day/max_temp_hour and min_temp_day/min_temp_hour to full UTC datetime.
    Adjusts times set to 24:00 to 00:00 of the following day.
    """
    # Get base date from data['date_time']
    base_date = data.get('date_time', None)
    if base_date is None:
        data['max_temperature_time'] = None
        data['min_temperature_time'] = None
        return data

    # Helper function to handle day/hour adjustments
    def calculate_temperature_time(day, hour):
        temp_time = datetime(
            year=base_date.year,
            month=base_date.month,
            day=int(day),
            hour=0 if hour == 24 else hour,
            tzinfo=base_date.tzinfo
        )
        # Shift to next day if hour was 24
        if hour == 24:
            temp_time += timedelta(days=1)
        # Handle month transitions if necessary
        if temp_time < base_date:
            temp_time += relativedelta(months=1)
        tempDate = calculate_temperature_dates(base_date, end_hour, end_hour)    
        valid_end_date = base_date.replace(year=tempDate.year, 
                                       day=tempDate.day, 
                                       hour=tempDate.hour, 
                                       minute=tempDate.minute, 
                                       second=tempDate.second)
        return temp_time

    # Process max temperature datetime
    max_day = data.get('max_temp_day')
    max_hour = int(data.get('max_temp_hour')) if data.get('max_temp_hour') else None
    data['max_temperature_time'] = calculate_temperature_time(max_day, max_hour) if max_day and max_hour is not None else None

    # Process min temperature datetime
    min_day = data.get('min_temp_day')
    min_hour = int(data.get('min_temp_hour')) if data.get('min_temp_hour') else None
    data['min_temperature_time'] = calculate_temperature_time(min_day, min_hour) if min_day and min_hour is not None else None

    return data


In [11]:
def process_probability(data):
    prob_str = data.get('probability', None)

    if prob_str and 'PROB' in prob_str:
        data['probability'] = round(float(data['prob_value']) / 100, 2)
        data['has_prob'] = True
    else:
        data['probability'] = 0.9
        data['has_prob'] = False
        
    return data


In [15]:
# Parse TAF report 
def parse_taf_block(date_time, line):
    """Parse a full TAF block into its components."""
    
    # Regex pattern to match main TAF components across multiple lines 
    match = taf_pattern.match(line)

    if match:
        data = match.groupdict()

        # Convert `date_time` to UTC format
        try:
            utc_date = datetime.strptime(date_time, "%Y/%m/%d %H:%M").replace(tzinfo=timezone.utc)
            data['date_time'] = utc_date.strftime("%Y-%m-%d %H:%M:%S")
            data['date_time'] = pd.to_datetime(data['date_time'], utc=True)
        except ValueError as e:
            data['date_time'] = None
        
        # Amended handling
        data['amended'] = True if data.get('amended') == 'AMD' else False

        # corection handling
        data['corection'] = True if data.get('corection') == 'COR' else False
        
        # validity date handling
        data = process_validity(data)

        data = process_wind(data)

        visibility = data.get('visibility')
        if visibility == "CAVOK":
            data['visibility_meters'] = 10000  # Convention for CAVOK
        elif visibility and "SM" in visibility:

            visibility_miles = float(visibility.replace("SM", ""))
            data['visibility_meters'] = int(visibility_miles * 1609.34)
        elif visibility and visibility.isdigit():
            data['visibility_meters'] = int(visibility)
        else:
            data['visibility_meters'] = None

        max_temp_value = data.get('max_temp_value', None)
        if max_temp_value is not None:
            value = re.search(r'\d+', max_temp_value)
            data['max_temp_value'] = int(max_temp_value) if 'M' not in max_temp_value else 0 - int(value.group())
            

        
        min_temp_value = data.get('min_temp_value', None)
        if min_temp_value is not None:
            value = re.search(r'\d+', min_temp_value)
            data['min_temp_value'] = int(min_temp_value) if 'M' not in min_temp_value else 0 - int(value.group())
        
        data = process_weather(data)

        clouds = data.get('clouds')
        data['cloud_layers'] = parse_cloud_layers(clouds.strip().split()) if clouds else []

        data = process_qnh(data)
        
        data = process_variable_wind(data)

        data = convert_temp_to_datetime(data)
        data = process_probability(data)
        fields_to_drop = ['valid_start_day', 'valid_start_hour', 'valid_end_hour', 'valid_end_day',
                           'wind', 'clouds', 'qnh', 'variable_wind', 'max_temp_day', 'max_temp_hour'
                           , 'min_temp_day', 'min_temp_hour', 'max_temp', 'min_temp', 'validity', 
                           'prob_value', 'visibility'] 
        for field in fields_to_drop:
            if field in data:
                del data[field]
        return data
    return None

In [16]:
file_path = '/home/jaosn/finalProject/data/TAF/train/taf.20220901.00Z.txt'

with open(file_path, 'r', encoding='ISO-8859-1') as file:
    lines = file.readlines()

data_entries = []
current_date_time = None
current_report_lines = []
for line in lines:
    line = line.strip()
    if not line:
        continue  # Skip empty lines
    # Check if the line contains a date-time
    date_time_match = re.match(r'\d{4}/\d{2}/\d{2} \d{2}:\d{2}', line)
    if date_time_match:
        # If there’s an accumulated report, parse it
        if current_report_lines:
            full_report = ' '.join(current_report_lines)
            # Split multiple TAF reports within the same block
            taf_reports = re.split(r'=\s*', full_report)
            for report in taf_reports:
                report = report.strip()
                if report:
                    report = remove_duplicates_in_report(report)
                    parsed_data = parse_taf_block(current_date_time, report)
                    if parsed_data:
                        data_entries.append(parsed_data)
            current_report_lines = []
        current_date_time = date_time_match.group()
    else:
        # Continue accumulating lines for the current report
        current_report_lines.append(line)

# Process the last report if any
if current_report_lines:
    full_report = ' '.join(current_report_lines)
    taf_reports = re.split(r'=\s*', full_report)
    for report in taf_reports:
        report = report.strip()
        if report:
            report = remove_duplicates_in_report(report)
            parsed_data = parse_taf_block(current_date_time, report)
            if parsed_data:
                data_entries.append(parsed_data)
df =pd.DataFrame(data_entries)

In [17]:
print(df)
print(df.dtypes)

      amended  corection station issue_datetime weather  max_temp_value  \
0       False      False    ZGKL       012104Z     None            32.0   
1       False      False    ZGNN       012108Z     None            35.0   
2       False      False    ZGOW       012101Z     None            35.0   
3       False      False    ZGSZ       012120Z     None            34.0   
4       False      False    ZLXY       012107Z       BR             NaN   
...       ...        ...     ...            ...     ...             ...   
1076    False      False    TLPL       012300Z     None             NaN   
1077    False      False    TLPC       012300Z     None             NaN   
1078    False      False    KNBG           None    None             NaN   
1079    False      False    KNHK           None     SKC             NaN   
1080    False      False    KNQX           None    None             NaN   

      min_temp_value  probability  \
0               20.0          0.9   
1               26.0     

In [19]:
import fetchData
taf_data = fetchData.load_data( 
    data_type="TAF", 
    dataset_purpose="test",  
    file_name="taf.20220925.00Z"
)
print(taf_data.dtypes)

amended                                bool
corection                              bool
station                              object
issue_datetime                       object
weather                              object
max_temp_value                      float64
min_temp_value                      float64
probability                         float64
additional_sections                  object
date_time               datetime64[ns, UTC]
valid_start_date        datetime64[ns, UTC]
valid_end_date          datetime64[ns, UTC]
has_wind                              int64
wind_is_variable                      int64
wind_direction                        int64
wind_speed_kt                       float64
wind_gust_kt                        float64
visibility_meters                   float64
weather_score                       float64
cloud_layers                         object
qnh_hpa                             float64
variable_wind_from                   object
variable_wind_to                